In [12]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [13]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [14]:
subject = 'São Paulo - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [15]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('São Paulo - Produção de Cimento (t)', axis=1)
data

,Ano,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade,São Paulo - Consumo de Cimento (t)
0,2008,0.809971,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,0.794724,5.483335e+08,2.546644e+09,1.874883e+10,...,8.071263,18.095190,836.334300,1204.525858,15.181901,11.136027,78.961939,1.773094,0.851565,13476.268000
1,2009,0.812524,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,0.793388,5.302764e+08,2.679945e+09,2.055015e+10,...,8.100327,17.457437,836.629901,1207.913300,15.173450,11.417327,80.559826,1.537503,0.853009,13304.301000
2,2010,0.814896,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,0.792439,5.046425e+08,2.891930e+09,2.342247e+10,...,8.064867,17.476576,836.656415,1207.430943,14.880006,11.288684,81.499868,1.367891,0.853860,14534.980000
3,2011,0.817083,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,0.791869,4.854622e+08,3.091988e+09,2.604265e+10,...,8.053175,17.341412,836.557224,1189.112243,14.815765,11.391381,82.321001,1.335640,0.854127,14871.943000
4,2012,0.818822,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,0.791406,4.867661e+08,3.189506e+09,2.708753e+10,...,10.058484,16.745399,836.685833,1175.350304,14.895124,11.497643,82.512436,1.341854,0.853708,15159.683000
5,2013,0.821057,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,0.792190,5.225848e+08,3.093872e+09,2.523394e+10,...,10.842573,15.620656,836.833718,1154.703253,15.020803,11.542975,82.278650,1.326810,0.852933,15089.297000
6,2014,0.822812,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,0.792622,5.676413e+08,2.779340e+09,1.954361e+10,...,11.415800,15.750987,836.986331,1108.545094,15.247907,11.986366,82.848722,1.532951,0.851595,13714.511286
7,2015,0.823629,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,0.792841,5.179392e+08,2.561731e+09,1.327596e+10,...,11.865646,15.893940,837.040042,1132.416417,15.520197,12.242333,82.064855,1.539659,0.849644,11767.653429
8,2016,0.832029,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,0.785642,4.524144e+08,2.516859e+09,1.011370e+10,...,11.874466,15.955560,837.011044,1181.481000,15.455320,11.972800,81.279861,1.362167,0.857989,9811.084286
9,2017,0.831577,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,0.785518,4.213689e+08,2.497710e+09,8.456309e+09,...,11.486716,16.984321,837.006952,1198.617570,15.304181,12.004953,82.751052,1.502384,0.855534,9918.501000


In [16]:
input_data = data.iloc[:-2, 1:-1]
input_data

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
0,0.809971,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,0.794724,5.483335e+08,2.546644e+09,1.874883e+10,2.351319e+10,...,0.818381,8.071263,18.095190,836.334300,1204.525858,15.181901,11.136027,78.961939,1.773094,0.851565
1,0.812524,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,0.793388,5.302764e+08,2.679945e+09,2.055015e+10,2.446518e+10,...,0.819150,8.100327,17.457437,836.629901,1207.913300,15.173450,11.417327,80.559826,1.537503,0.853009
2,0.814896,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,0.792439,5.046425e+08,2.891930e+09,2.342247e+10,2.606035e+10,...,0.819803,8.064867,17.476576,836.656415,1207.430943,14.880006,11.288684,81.499868,1.367891,0.853860
3,0.817083,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,0.791869,4.854622e+08,3.091988e+09,2.604265e+10,2.845738e+10,...,0.820344,8.053175,17.341412,836.557224,1189.112243,14.815765,11.391381,82.321001,1.335640,0.854127
4,0.818822,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,0.791406,4.867661e+08,3.189506e+09,2.708753e+10,3.181492e+10,...,0.820550,10.058484,16.745399,836.685833,1175.350304,14.895124,11.497643,82.512436,1.341854,0.853708
5,0.821057,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,0.792190,5.225848e+08,3.093872e+09,2.523394e+10,3.629162e+10,...,0.821262,10.842573,15.620656,836.833718,1154.703253,15.020803,11.542975,82.278650,1.326810,0.852933
6,0.822812,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,0.792622,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,0.821534,11.415800,15.750987,836.986331,1108.545094,15.247907,11.986366,82.848722,1.532951,0.851595
7,0.823629,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,0.792841,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,0.821245,11.865646,15.893940,837.040042,1132.416417,15.520197,12.242333,82.064855,1.539659,0.849644
8,0.832029,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,0.785642,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,0.824327,11.874466,15.955560,837.011044,1181.481000,15.455320,11.972800,81.279861,1.362167,0.857989
9,0.831577,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,0.785518,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,0.823356,11.486716,16.984321,837.006952,1198.617570,15.304181,12.004953,82.751052,1.502384,0.855534


In [17]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     13304.301000
1     14534.980000
2     14871.943000
3     15159.683000
4     15089.297000
5     13714.511286
6     11767.653429
7      9811.084286
8      9918.501000
9     10001.077000
10    10458.276000
11    11181.809000
12    12113.838000
13    12383.649000
14             NaN
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [18]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
0,-1.601914,-1.819408,1.003478,1.202107,-1.961368,1.285446,1.204752,-0.990234,0.166091,-1.487840,...,-1.705828,-1.355043,1.636748,-2.157967,0.548623,0.114751,-1.578154,-2.418314,2.112223,-0.575401
1,-1.272998,-1.675567,1.209427,1.179772,-1.951541,0.894167,0.820657,-0.546688,0.428911,-1.378556,...,-1.234652,-1.336507,0.960381,-0.797175,0.643440,0.075193,-0.842756,-1.158674,0.402706,0.032912
2,-0.967315,-1.094542,1.324177,1.230212,-0.558611,0.616212,0.275392,0.158672,0.847995,-1.195437,...,-0.834215,-1.359122,0.980679,-0.675119,0.629939,-1.298380,-1.179065,-0.417623,-0.828049,0.391069
3,-0.685507,-0.575726,1.019683,1.107266,0.259840,0.449046,-0.132596,0.824346,1.230291,-0.920269,...,-0.503056,-1.366579,0.837331,-1.131742,0.117185,-1.599084,-0.910586,0.229689,-1.062071,0.503752
4,-0.461531,-0.115211,0.708407,0.661426,0.781023,0.313381,-0.104860,1.148829,1.382743,-0.534838,...,-0.376604,-0.087675,0.205232,-0.539690,-0.268021,-1.227616,-0.632790,0.380601,-1.016982,0.327130
5,-0.173555,0.286253,0.327534,0.184658,1.145924,0.543033,0.657046,0.830615,1.112297,-0.020933,...,0.060098,0.412385,-0.987611,0.141096,-0.845947,-0.639328,-0.514278,0.196304,-1.126146,0.001028
6,0.052526,0.597984,-0.189503,-0.219884,1.082248,0.669744,1.615453,-0.215960,0.282052,0.569384,...,0.226730,0.777966,-0.849388,0.843646,-2.137947,0.423719,0.644870,0.645700,0.369675,-0.562814
7,0.157871,0.806234,-0.656965,-0.723606,0.683738,0.733766,0.558230,-0.940033,-0.632425,0.943972,...,0.049709,1.064859,-0.697781,1.090901,-1.469772,1.698271,1.314042,0.027766,0.418348,-1.384206
8,1.240079,0.941752,-1.045917,-1.009627,0.458458,-1.375468,-0.835563,-1.089341,-1.093812,1.140087,...,1.938197,1.070484,-0.632429,0.957410,-0.096419,1.394591,0.609405,-0.591059,-0.869586,2.130112
9,1.181857,0.973717,-1.232918,-1.125922,0.321024,-1.411648,-1.495939,-1.153056,-1.335633,1.102599,...,1.343468,0.823193,0.458620,0.938576,0.383246,0.687130,0.693462,0.568705,0.147869,1.096015


In [19]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     13304.301000
1     14534.980000
2     14871.943000
3     15159.683000
4     15089.297000
5     13714.511286
6     11767.653429
7      9811.084286
8      9918.501000
9     10001.077000
10    10458.276000
11    11181.809000
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [20]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
12,1.001135,0.409525,-0.926633,-1.322522,-1.033191,-1.385738,1.108405,2.635823,-0.189959,0.726405,...,-1.092902,0.514165,-0.901047,-0.751547,0.232288,-0.363994,-0.503159,-0.998268,-1.560012,-2.468148


In [21]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    12113.838
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [22]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [23]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [25]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2426021178, 999018374, 862905778, 2553661660, 2031517046, 508204441, 3469916650, 2673373398, 2569848023, 1721744248, 3168810259, 3640993363, 1381663125, 2638650672, 2257577640, 584785252, 3676215991, 269549089, 3107607944, 2208627332, 3946253315, 3250292145, 2044352209, 166130938, 3895960218, 4067039141, 1234640998, 238455293, 3585708784, 659521079, 3167478353, 305149301, 2109592177, 2322492930, 1140619143, 3158105487, 1275541807, 3186752755, 3799985195, 3556095903, 4094704592, 3536424292, 448049385, 66857829, 489487193, 1310875933, 998789036, 3053400500, 4274398797, 484430478, 905809194, 3067637811, 2412521915, 899195837, 2356482947, 3505695543, 171946101, 1556633868, 1201154981, 4105074581, 3752053263, 202143407, 2087629551, 545042440, 1945450878, 2602862469, 1513257308, 4192509669, 3193714367, 1436564039, 3248017956, 2943261647, 2757279226, 291628446, 3472650168, 3331278710, 65730824, 1231735511, 3331456464, 2885839283, 2574498443, 3867819746, 2629602998, 1172258148, 2719563414, 31

val_loss: 359088.84375


Step: 89 ___________________________________________
val_loss: 294275.53125


Step: 90 ___________________________________________
val_loss: 252669.203125


Step: 91 ___________________________________________
val_loss: 96745.296875


Step: 92 ___________________________________________
val_loss: 2608.675537109375


Step: 93 ___________________________________________
val_loss: 157384.984375


Step: 94 ___________________________________________
val_loss: 358824.40625


Step: 95 ___________________________________________
val_loss: 168138.4375


Step: 96 ___________________________________________
val_loss: 40302.1328125


Step: 97 ___________________________________________
val_loss: 0.00034427642822265625
winner_seed: 2446989432


Step: 98 ___________________________________________
val_loss: 345434.0


Step: 99 ___________________________________________
val_loss: 17379.94140625


final_seed: 2446989432


In [26]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 492ms/step - loss: 160210192.0000 - val_loss: 129997544.0000
Epoch 2/10000
1/1 [==============================] - 0s 16ms/step - loss: 152005840.0000 - val_loss: 16270796.0000
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 231169360.0000 - val_loss: 190678800.0000
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 140719888.0000 - val_loss: 186487456.0000
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 135730944.0000 - val_loss: 169866464.0000
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 140892208.0000 - val_loss: 193958720.0000
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 147113744.0000 - val_loss: 119499592.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 141694608.0000 - val_loss: 196872848.0000
Epoch 9/10000
1/1 [==============================] - 0s 16ms/step - loss

Epoch 72/10000
1/1 [==============================] - 0s 14ms/step - loss: 36025132.0000 - val_loss: 24592872.0000
Epoch 73/10000
1/1 [==============================] - 0s 14ms/step - loss: 33843500.0000 - val_loss: 25454878.0000
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 39140820.0000 - val_loss: 30330394.0000
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 36198808.0000 - val_loss: 39734728.0000
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 34782028.0000 - val_loss: 32397842.0000
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 33951364.0000 - val_loss: 31604304.0000
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 32373072.0000 - val_loss: 22337172.0000
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 30278970.0000 - val_loss: 20031368.0000
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 36034

Epoch 143/10000
1/1 [==============================] - 0s 15ms/step - loss: 8963343.0000 - val_loss: 14151013.0000
Epoch 144/10000
1/1 [==============================] - 0s 14ms/step - loss: 10673601.0000 - val_loss: 15171984.0000
Epoch 145/10000
1/1 [==============================] - 0s 14ms/step - loss: 9890582.0000 - val_loss: 16832674.0000
Epoch 146/10000
1/1 [==============================] - 0s 14ms/step - loss: 8545565.0000 - val_loss: 17503282.0000
Epoch 147/10000
1/1 [==============================] - 0s 14ms/step - loss: 8996417.0000 - val_loss: 19352280.0000
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 9658709.0000 - val_loss: 16686850.0000
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 9040903.0000 - val_loss: 17296124.0000
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 8556867.0000 - val_loss: 13191119.0000
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 721

Epoch 215/10000
1/1 [==============================] - 0s 14ms/step - loss: 2754468.2500 - val_loss: 2944307.5000
Epoch 216/10000
1/1 [==============================] - 0s 14ms/step - loss: 2810670.2500 - val_loss: 3128341.7500
Epoch 217/10000
1/1 [==============================] - 0s 14ms/step - loss: 2679359.7500 - val_loss: 3233320.2500
Epoch 218/10000
1/1 [==============================] - 0s 14ms/step - loss: 2433509.2500 - val_loss: 2964127.2500
Epoch 219/10000
1/1 [==============================] - 0s 14ms/step - loss: 2508708.0000 - val_loss: 2890478.2500
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 2529287.7500 - val_loss: 3384234.5000
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 2304547.7500 - val_loss: 3147010.2500
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 2437259.7500 - val_loss: 2855403.5000
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 2625472.0000

1/1 [==============================] - 0s 14ms/step - loss: 1284496.3750 - val_loss: 418476.3125
Epoch 288/10000
1/1 [==============================] - 0s 14ms/step - loss: 854796.5000 - val_loss: 684681.9375
Epoch 289/10000
1/1 [==============================] - 0s 14ms/step - loss: 802418.0000 - val_loss: 595444.3125
Epoch 290/10000
1/1 [==============================] - 0s 14ms/step - loss: 792780.7500 - val_loss: 583147.0000
Epoch 291/10000
1/1 [==============================] - 0s 14ms/step - loss: 795766.5625 - val_loss: 577512.4375
Epoch 292/10000
1/1 [==============================] - 0s 14ms/step - loss: 768037.3125 - val_loss: 542343.0625
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 759529.0625 - val_loss: 529178.1250
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 751681.4375 - val_loss: 503000.9375
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 975972.8125 - val_loss: 458515.4688
Epoch 2

...

1/1 [==============================] - 0s 14ms/step - loss: 425971.2500 - val_loss: 1904.1632
Epoch 1470/10000
1/1 [==============================] - 0s 14ms/step - loss: 412833.6250 - val_loss: 210.5333
Epoch 1471/10000
1/1 [==============================] - 0s 14ms/step - loss: 438855.1250 - val_loss: 954.8945
Epoch 1472/10000
1/1 [==============================] - 0s 14ms/step - loss: 464260.3438 - val_loss: 3577.0679
Epoch 1473/10000
1/1 [==============================] - 0s 14ms/step - loss: 472818.0938 - val_loss: 2046.0603
Epoch 1474/10000
1/1 [==============================] - 0s 14ms/step - loss: 455188.7500 - val_loss: 475.4444
Epoch 1475/10000
1/1 [==============================] - 0s 14ms/step - loss: 464323.4062 - val_loss: 3029.4060
Epoch 1476/10000
1/1 [==============================] - 0s 14ms/step - loss: 426233.7500 - val_loss: 618.7656
Epoch 1477/10000
1/1 [==============================] - 0s 14ms/step - loss: 471046.3750 - val_loss: 509.3308
Epoch 1478/10000
1/1 [=

1/1 [==============================] - 0s 15ms/step - loss: 526840.0000 - val_loss: 4060.8259
Epoch 1544/10000
1/1 [==============================] - 0s 15ms/step - loss: 433004.0000 - val_loss: 534.2238
Epoch 1545/10000
1/1 [==============================] - 0s 14ms/step - loss: 437909.2500 - val_loss: 1739.1506
Epoch 1546/10000
1/1 [==============================] - 0s 14ms/step - loss: 416264.5000 - val_loss: 1140.9089
Epoch 1547/10000
1/1 [==============================] - 0s 14ms/step - loss: 436504.7812 - val_loss: 8208.2891
Epoch 1548/10000
1/1 [==============================] - 0s 14ms/step - loss: 438066.4062 - val_loss: 3356.9802
Epoch 1549/10000
1/1 [==============================] - 0s 14ms/step - loss: 434172.1562 - val_loss: 7853.1792
Epoch 1550/10000
1/1 [==============================] - 0s 14ms/step - loss: 419988.1562 - val_loss: 1368.2775
Epoch 1551/10000
1/1 [==============================] - 0s 14ms/step - loss: 484683.2500 - val_loss: 843.2672
Epoch 1552/10000
1/1

1/1 [==============================] - 0s 14ms/step - loss: 410529.9062 - val_loss: 285.2932
Epoch 1618/10000
1/1 [==============================] - 0s 14ms/step - loss: 436789.9688 - val_loss: 5157.5962
Epoch 1619/10000
1/1 [==============================] - 0s 14ms/step - loss: 418238.8438 - val_loss: 54.6502
Epoch 1620/10000
1/1 [==============================] - 0s 14ms/step - loss: 429596.8438 - val_loss: 714.1003
Epoch 1621/10000
1/1 [==============================] - 0s 14ms/step - loss: 412758.0938 - val_loss: 672.6019
Epoch 1622/10000
1/1 [==============================] - 0s 15ms/step - loss: 412724.9062 - val_loss: 14.0113
Epoch 1623/10000
1/1 [==============================] - 0s 14ms/step - loss: 502826.2188 - val_loss: 579.7091
Epoch 1624/10000
1/1 [==============================] - 0s 14ms/step - loss: 435485.0938 - val_loss: 20.4085
Epoch 1625/10000
1/1 [==============================] - 0s 14ms/step - loss: 410493.7188 - val_loss: 135.6633
Epoch 1626/10000
1/1 [=======

1/1 [==============================] - 0s 15ms/step - loss: 440759.2188 - val_loss: 1766.4618
Epoch 1692/10000
1/1 [==============================] - 0s 15ms/step - loss: 485258.9062 - val_loss: 7174.7847
Epoch 1693/10000
1/1 [==============================] - 0s 14ms/step - loss: 439005.6562 - val_loss: 204.1215
Epoch 1694/10000
1/1 [==============================] - 0s 14ms/step - loss: 464791.0938 - val_loss: 1922.2744
Epoch 1695/10000
1/1 [==============================] - 0s 14ms/step - loss: 456111.9062 - val_loss: 118.2444
Epoch 1696/10000
1/1 [==============================] - 0s 14ms/step - loss: 413188.5000 - val_loss: 771.6351
Epoch 1697/10000
1/1 [==============================] - 0s 14ms/step - loss: 414076.5000 - val_loss: 39.9464
Epoch 1698/10000
1/1 [==============================] - 0s 14ms/step - loss: 411224.0000 - val_loss: 126.4966
Epoch 1699/10000
1/1 [==============================] - 0s 14ms/step - loss: 462730.7500 - val_loss: 1456.7192
Epoch 1700/10000
1/1 [==

1/1 [==============================] - 0s 14ms/step - loss: 424361.0000 - val_loss: 5216.5352
Epoch 1766/10000
1/1 [==============================] - 0s 14ms/step - loss: 426936.4062 - val_loss: 7291.0586
Epoch 1767/10000
1/1 [==============================] - 0s 14ms/step - loss: 423235.2188 - val_loss: 4221.9536
Epoch 1768/10000
1/1 [==============================] - 0s 14ms/step - loss: 450454.1562 - val_loss: 283.1201
Epoch 1769/10000
1/1 [==============================] - 0s 14ms/step - loss: 409975.7812 - val_loss: 50811.0586
Epoch 1770/10000
1/1 [==============================] - 0s 14ms/step - loss: 423418.8438 - val_loss: 75993.9062
Epoch 1771/10000
1/1 [==============================] - 0s 14ms/step - loss: 415476.5938 - val_loss: 71226.4375
Epoch 1772/10000
1/1 [==============================] - 0s 14ms/step - loss: 464793.1562 - val_loss: 39637.5430
Epoch 1773/10000
1/1 [==============================] - 0s 14ms/step - loss: 420213.5312 - val_loss: 73066.7344
Epoch 1774/100

In [27]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 50ms/step
[12113.838](test_target) - [[11181.827]](prediction) = 932.0108515624997


In [28]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [29]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
0,-1.239414,-0.924099,-1.323126,-0.093200,-0.714550,1.287736,1.148205,-1.122043,-1.121418,-1.109924,...,-1.256499,-0.489673,1.413722,-1.410322,-1.401911,0.736997,-1.258070,-1.315911,1.285551,-1.315803
1,0.029885,-0.465059,0.229123,-1.175483,-0.699637,-0.137605,0.140893,-0.184474,-0.185492,-0.204018,...,0.066193,1.393821,-0.739164,0.614359,0.862152,0.676789,1.188450,0.209310,-0.132366,0.209017
2,1.209529,1.389158,1.094002,1.268682,1.414187,-1.150131,-1.289098,1.306518,1.306910,1.313942,...,1.190306,-0.904148,-0.674558,0.795962,0.539759,-1.413786,0.069619,1.106602,-1.153185,1.106786


train_target:


,São Paulo - Consumo de Cimento (t)
0,13304.301
1,14534.980
2,14871.943


test_input:


...

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
12,1.001135,0.409525,-0.926633,-1.322522,-1.033191,-1.385738,1.108405,2.635823,-0.189959,0.726405,...,-1.092902,0.514165,-0.901047,-0.751547,0.232288,-0.363994,-0.503159,-0.998268,-1.560012,-2.468148


test_target:


,São Paulo - Consumo de Cimento (t)
12,12113.838


1/1 [==============================] - 0s 34ms/step
Error: 932.0108515624997




[14871.9296875,
 15160.77734375,
 15048.138671875,
 12743.6572265625,
 12108.263671875,
 9788.966796875,
 9918.998046875,
 9998.7607421875,
 10458.201171875,
 11181.8271484375]

In [30]:
display(targets)
display(predictions)

[15159.683,
 15089.297,
 13714.511285714289,
 11767.65342857143,
 9811.084285714289,
 9918.501,
 10001.077,
 10458.276,
 11181.809,
 12113.838]

[14871.9296875,
 15160.77734375,
 15048.138671875,
 12743.6572265625,
 12108.263671875,
 9788.966796875,
 9918.998046875,
 9998.7607421875,
 10458.201171875,
 11181.8271484375]

In [31]:
mae = mean_absolute_error(predictions, targets)
mae

729.2791320312492

In [32]:
porcentage = mae/np.mean(targets)
porcentage

0.061173062651317